In [1]:
%load_ext autoreload
%autoreload 2

import spectral_unmixing_tools as el_spectral

In [44]:
pip install git+https://github.com/earthlab/cross-sensor-cal.git


  Cloning https://github.com/earthlab/cross-sensor-cal.git to /tmp/pip-req-build-3mcb9iw1
  Running command git clone --filter=blob:none --quiet https://github.com/earthlab/cross-sensor-cal.git /tmp/pip-req-build-3mcb9iw1
  Resolved https://github.com/earthlab/cross-sensor-cal.git to commit 8717d6b99deb35c43dc57fc2bec4882c2f254814
  Preparing metadata (setup.py) ... done
  Created wheel for EarthLabSpectral: filename=EarthLabSpectral-0.1-py3-none-any.whl size=1417 sha256=bebfc882dd13f60e077c2b288b2ae7dd343eae9db8bd57580f13cd335263fe64
  Stored in directory: /tmp/pip-ephem-wheel-cache-e_8ou5jp/wheels/d4/97/e8/c93ae5a8364c661ba5327b524ebe4ee01e41cfe582ac66f3d0
Successfully built EarthLabSpectral
Note: you may need to restart the kernel to use updated packages.


In [14]:
base_folder = "NIWOT_calibration_flight_08_2020"
el_spectral.resample_translation_to_other_sensors(base_folder)

Processing folder: NIWOT_calibration_flight_08_2020/NEON_D13_NIWO_DP1_20200801_161441_reflectance
Command executed successfully for sensor type: Landsat 5 TM
Standard Output: {'description': 'Resampled hyperspectral data', 'file type': 'ENVI Standard', 'sensor type': 'landsat', 'bands': 6, 'lines': '10782', 'samples': '1071', 'header offset': 0, 'data type': 4, 'interleave': 'bil', 'byte order': 0, 'wavelength units': 'Nanometers', 'wavelength': array([ 485.,  560.,  660.,  830., 1650., 2215.], dtype=float32)}

Command executed successfully for sensor type: Landsat 7 ETM+
Standard Output: {'description': 'Resampled hyperspectral data', 'file type': 'ENVI Standard', 'sensor type': 'landsat', 'bands': 6, 'lines': '10782', 'samples': '1071', 'header offset': 0, 'data type': 4, 'interleave': 'bil', 'byte order': 0, 'wavelength units': 'Nanometers', 'wavelength': array([ 477.,  560.,  662.,  835., 1648., 2206.], dtype=float32)}

Command executed successfully for sensor type: Landsat 8 OLI
S

## Extract spectra as array

In [ ]:
import rasterio
import numpy as np

class ENVIProcessor:
    def __init__(self, file_path):
        self.file_path = file_path
        self.data = None  # This will hold the raster data array
        self.file_type = "envi"
        # Other attributes...

    def load_data(self):
        """Loads the raster data from the file_path into self.data"""
        with rasterio.open(self.file_path) as src:
            self.data = src.read()  # Read all bands

    def get_chunk_from_extent(self, corrections=[], resample=False):

        self.load_data()  # Ensure data is loaded
        with rasterio.open(self.file_path) as src:
            bounds = src.bounds
            width, height = src.width, src.height
            col_start, line_start = 0, 0
            col_end, line_end = width, height

            # Assuming self.data is a 3D numpy array with dimensions [bands, rows, cols]
            chunk = self.data[:, line_start:line_end, col_start:col_end]
            
            # Apply any processing to chunk here...
            # For example, to demonstrate, flip chunk vertically
            chunk = np.flip(chunk, axis=1)
            
            return chunk



# Example usage:
# Replace 'your_envi_file_path' with the actual path to your ENVI file
raster_path_5 = "NIWOT_calibration_flight_08_2020/NEON_D13_NIWO_DP1_20200801_161441_reflectance/NEON_D13_NIWO_DP1_20200801_161441_reflectanceNEON_D13_NIWO_DP1_20200801_161441_reflectance__envi_resample_Landsat_5_TM.img"
raster_path_7 = "NIWOT_calibration_flight_08_2020/NEON_D13_NIWO_DP1_20200801_161441_reflectance/NEON_D13_NIWO_DP1_20200801_161441_reflectanceNEON_D13_NIWO_DP1_20200801_161441_reflectance__envi_resample_Landsat_5_TM.img"
raster_path_8 = "NIWOT_calibration_flight_08_2020/NEON_D13_NIWO_DP1_20200801_161441_reflectance/NEON_D13_NIWO_DP1_20200801_161441_reflectanceNEON_D13_NIWO_DP1_20200801_161441_reflectance__envi_resample_Landsat_5_TM.img"
raster_path_9 = "NIWOT_calibration_flight_08_2020/NEON_D13_NIWO_DP1_20200801_161441_reflectance/NEON_D13_NIWO_DP1_20200801_161441_reflectanceNEON_D13_NIWO_DP1_20200801_161441_reflectance__envi_resample_Landsat_5_TM.img"
raster_path_corrected = "NIWOT_calibration_flight_08_2020/NEON_D13_NIWO_DP1_20200801_161441_reflectance/NEON_D13_NIWO_DP1_20200801_161441_reflectanceNEON_D13_NIWO_DP1_20200801_161441_reflectance__envi"
raster_path_original = "NIWOT_calibration_flight_08_2020/NEON_D13_NIWO_DP1_20200801_161441_reflectance/NEON_D13_NIWO_DP1_20200801_161441_reflectance"



processor = ENVIProcessor(raster_path_5)
chunk_5 = processor.get_chunk_from_extent(corrections=['some_correction'], resample=False)
processor = ENVIProcessor(raster_path_7)
chunk_7 = processor.get_chunk_from_extent(corrections=['some_correction'], resample=False)
processor = ENVIProcessor(raster_path_8)
chunk_8 = processor.get_chunk_from_extent(corrections=['some_correction'], resample=False)
processor = ENVIProcessor(raster_path_9)
chunk_9 = processor.get_chunk_from_extent(corrections=['some_correction'], resample=False)

processor = ENVIProcessor(raster_path_corrected)
chunk_corrected = processor.get_chunk_from_extent(corrections=['some_correction'], resample=False)
processor = ENVIProcessor(raster_path_original)
chunk_original = processor.get_chunk_from_extent(corrections=['some_correction'], resample=False)

#combined_array = np.concatenate((chunk_corrected, chunk_5, chunk_7, chunk_8, chunk_9), axis=0)



In [ ]:
combined_array = np.concatenate((chunk_corrected, chunk_original, chunk_5, chunk_7, chunk_8, chunk_9), axis=0)
combined_array.shape


## flatten array into a 2D df


In [ ]:
import pandas as pd
import numpy as np

def flatten_array_to_dataframe(array):
    """
    Flattens a 3D numpy array into a pandas DataFrame.
    
    Parameters:
    - array: A 3D numpy array of shape (bands, rows, cols).
    
    Returns:
    - A pandas DataFrame where each row represents a pixel across all bands.
    """
    # Ensure the input is a 3D numpy array
    if len(array.shape) != 3:
        raise ValueError("Input array must be 3-dimensional.")
    
    bands, rows, cols = array.shape
    # Reshape the array to have pixels as rows and bands as columns
    reshaped_array = array.reshape(bands, -1).T  # Transpose to make bands as columns
    
    # Create a DataFrame from the reshaped array
    df = pd.DataFrame(reshaped_array, columns=[f'Band_{i+1}' for i in range(bands)])
    
    # Optionally, add pixel row and column indices
    pixel_indices = np.indices((rows, cols)).reshape(2, -1).T  # 2D array of row,col indices for each pixel
    df['Pixel_Row'] = pixel_indices[:, 0]
    df['Pixel_Col'] = pixel_indices[:, 1]
    
    return df

# Example usage:
# Assuming 'chunk' is your 3D numpy array of shape (426, 11138, 1031)

#df_5 = flatten_array_to_dataframe(chunk_5)
df_corrected = flatten_array_to_dataframe(combined_array)
df_corrected
columns_order = ['Pixel_Row', 'Pixel_Col'] + [col for col in df_corrected.columns if col not in ['Pixel_Row', 'Pixel_Col']]
df_corrected = df_corrected[columns_order]


In [ ]:
df_corrected

In [ ]:
import pandas as pd

# Assuming df_corrected is your input DataFrame

# Calculate the expected new names count (for "Band_" columns).
expected_new_names_count = 426 * 2 + 4 * 6  # Original + Corrected + 4 Landsat versions * 6

# Verify the number of Band_ columns matches the expected count for renaming.
if len([col for col in df_corrected.columns if col.startswith('Band_')]) == expected_new_names_count:
    # Generate new column names.
    new_column_names = []
    new_column_names.extend([f"Original_band_{i}" for i in range(1, 427)])
    new_column_names.extend([f"Corrected_band_{i}" for i in range(1, 427)])
    for landsat_version in [5, 7, 8, 9]:  # Skipping Landsat 6 as per your note
        new_column_names.extend([f"Landsat_{landsat_version}_band_{i}" for i in range(1, 7)])
    
    # Identify "Band_" columns.
    band_columns = [col for col in df_corrected.columns if col.startswith('Band_')]
    
    # Create a mapping from old 'Band_' columns to new names.
    # Instead of creating a new DataFrame, rename columns in place.
    for old_name, new_name in zip(band_columns, new_column_names):
        df_corrected.rename(columns={old_name: new_name}, inplace=True)

    print("Renaming completed. The DataFrame has been updated in place.")

    # Since the renaming is done in place, we can simply assign df_corrected to spectra_per_pixel.
    spectra_per_pixel = df_corrected
else:
    print("Mismatch in the total expected 'Band_' column count. Please verify the DataFrame structure.")


In [ ]:
# Assuming 'spectra_per_pixel' is your DataFrame

# Step 1: Create the 'Pixel_id' column by copying the index
# This step is essentially just preparation, no heavy memory operation is involved here
pixel_id_series = spectra_per_pixel.index.to_series()

# Step 2: Insert the 'Pixel_id' column at position 0
# The .insert() method is more memory efficient for this operation
spectra_per_pixel.insert(0, 'Pixel_id', pixel_id_series)

# Now, 'spectra_per_pixel' has 'Pixel_id' as its first column, without extensive memory overhead.


spectra_per_pixel

In [ ]:
import pandas as pd
import numpy as np

def clean_data(df):
    """
    Cleans the input DataFrame by replacing values approximately equal to -9999
    (within a tolerance of 1) with NaN, and then dropping rows where all layer column values
    are NaN, excluding columns that contain 'Pixel' in their name.
    
    Parameters:
    - df: pandas DataFrame to clean.
    
    Returns:
    - A cleaned pandas DataFrame.
    """
    # Function to replace approx -9999 with NaN
    def replace_approx_with_nan(x):
        if isinstance(x, (int, float)) and np.isclose(x, -9999, atol=1):
            return np.nan
        else:
            return x

    # Apply the function across the DataFrame
    df = df.applymap(replace_approx_with_nan)
    
    # Define the columns to consider for NaN checking, assuming 'Band_' prefix
    layer_columns = [f'Band_{i}' for i in range(1, 426)]
    
    # Assuming 'Pixel' columns are not among the 'layer_columns', they won't be affected
    # Drop rows where all values in the specified columns are NaN
    df_cleaned = df.dropna(subset=layer_columns, how='all')
    
    return df_cleaned


# Apply the function to your DataFrame
active_pixels = clean_data(spectra_per_pixel)
active_pixels

In [69]:
layer_columns = [f'Band_{i}' for i in range(1, 6)]
df_5 = df_5.replace(-9999.0, np.nan)
# Drop rows where all values in the specified columns are NaN
df_cleaned_5 = df_5.dropna(subset=layer_columns, how='any')
df_cleaned_5

,Band_1,Band_2,Band_3,Band_4,Band_5,Band_6,Pixel_Row,Pixel_Col
601,176.467301,180.005216,176.912213,506.815147,507.947371,343.149584,0,601
602,131.119576,142.267092,144.794049,500.456375,460.842812,273.245732,0,602
603,259.312357,341.406125,358.740937,1266.080438,851.560919,516.889035,0,603
604,348.415065,463.406689,515.622892,1648.415196,1224.635598,774.331352,0,604
605,232.145143,311.482266,355.386529,1124.511298,1049.291871,679.454621,0,605
...,...,...,...,...,...,...,...,...
11547457,97.941469,141.358131,121.724182,916.418683,352.889956,155.776193,10781,1006
11547458,136.444084,194.532201,177.253087,935.621082,491.615976,261.720447,10781,1007
11547459,128.240400,189.185338,173.412447,857.040145,489.123432,261.120619,10781,1008
11547460,109.851099,153.761957,139.001058,782.641279,425.945568,215.723224,10781,1009


## Reshape data frame and change labels

In [55]:
import numpy as np

def iterate_flatten_melt_array(array):
    """
    Generator to iterate over a 3D numpy array and yield "melted" data.
    
    Parameters:
    - array: A 3D numpy array of shape (bands, rows, cols).
    
    Yields:
    - Tuple of (Pixel_Row, Pixel_Col, Band_ID, Wavelength) for each pixel-band combination.
    """
    bands, rows, cols = array.shape
    
    for band in range(bands):
        for row in range(rows):
            for col in range(cols):
                yield (row, col, f'Band_{band+1}', array[band, row, col])

# Example usage
#chunk = np.random.rand(426, 11138, 1031)  # Replace with your actual data

# To demonstrate or test the generator, you can iterate through a small portion of it
for i, data_point in enumerate(iterate_flatten_melt_array(chunk)):
    print(data_point)
    if i > 100:  # Adjust this condition to control how many items you want to print
        break


(0, 0, 'Band_1', -9998.999997820287)
(0, 1, 'Band_1', -9998.999997820287)
(0, 2, 'Band_1', -9998.999997820287)
(0, 3, 'Band_1', -9998.999997820287)
(0, 4, 'Band_1', -9998.999997820287)
(0, 5, 'Band_1', -9998.999997820287)
(0, 6, 'Band_1', -9998.999997820287)
(0, 7, 'Band_1', -9998.999997820287)
(0, 8, 'Band_1', -9998.999997820287)
(0, 9, 'Band_1', -9998.999997820287)
(0, 10, 'Band_1', -9998.999997820287)
(0, 11, 'Band_1', -9998.999997820287)
(0, 12, 'Band_1', -9998.999997820287)
(0, 13, 'Band_1', -9998.999997820287)
(0, 14, 'Band_1', -9998.999997820287)
(0, 15, 'Band_1', -9998.999997820287)
(0, 16, 'Band_1', -9998.999997820287)
(0, 17, 'Band_1', -9998.999997820287)
(0, 18, 'Band_1', -9998.999997820287)
(0, 19, 'Band_1', -9998.999997820287)
(0, 20, 'Band_1', -9998.999997820287)
(0, 21, 'Band_1', -9998.999997820287)
(0, 22, 'Band_1', -9998.999997820287)
(0, 23, 'Band_1', -9998.999997820287)
(0, 24, 'Band_1', -9998.999997820287)
(0, 25, 'Band_1', -9998.999997820287)
(0, 26, 'Band_1', -999

In [ ]:
import csv

# Open a CSV file for writing
with open('melted_data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Pixel_Row', 'Pixel_Col', 'Band_ID', 'Wavelength'])  # Write header

    # Write each data point
    for data_point in iterate_flatten_melt_array(chunk):
        writer.writerow(data_point)
import numpy as np
import pandas as pd

def batch_flatten_melt_array(array, batch_size=1000000):
    """
    Generator to iterate over a 3D numpy array and yield batches of "melted" data.
    
    Parameters:
    - array: A 3D numpy array of shape (bands, rows, cols).
    - batch_size: The number of rows in each batch.
    
    Yields:
    - A DataFrame containing a batch of melted data.
    """
    bands, rows, cols = array.shape
    total_pixels = rows * cols
    num_batches = (total_pixels + batch_size - 1) // batch_size  # Ceiling division to get the number of batches
    
    for batch in range(num_batches):
        batch_data = []
        start_index = batch * batch_size
        end_index = min(start_index + batch_size, total_pixels)
        
        for index in range(start_index, end_index):
            row = index // cols
            col = index % cols
            for band in range(bands):
                batch_data.append((row, col, f'Band_{band+1}', array[band, row, col]))
                
        batch_df = pd.DataFrame(batch_data, columns=['Pixel_Row', 'Pixel_Col', 'Band_ID', 'Wavelength'])
        yield batch_df

# Example usage
#chunk = np.random.rand(426, 11138, 1031)  # Replace with your actual data

# Iterate through each batch and process
for i, batch_df in enumerate(batch_flatten_melt_array(chunk)):
    print(f"Processing batch {i+1}")
    # Process the batch_df here
    # For example, you could save each batch to a separate CSV file
    batch_df.to_csv(f'melted_data_batch_{i+1}.csv', index=False)
   # if i == 0:  # For demonstration, break after processing the first batch
    #    break


## Exract by polygon

In [7]:
import rasterio
import numpy as np
import geopandas as gpd
from rasterio.windows import from_bounds
from shapely.geometry import box

class ENVIProcessor:
    def __init__(self, raster_path, polygons_path):
        self.raster_path = raster_path
        self.polygons_path = polygons_path
        self.polygons = None
        self.raster_meta = None
        
    def load_polygons(self):
        """Loads the polygons and ensures they are in the same CRS as the raster."""
        with rasterio.open(self.raster_path) as src:
            self.raster_meta = src.meta
            self.polygons = gpd.read_file(self.polygons_path)
            self.polygons = self.polygons.to_crs(src.crs)
    
    def extract_data_by_polygons(self):
        """Extracts chunks from the raster based on polygons and concatenates them."""
        self.load_polygons()  # Load polygons and ensure CRS match
        
        all_data = []
        with rasterio.open(self.raster_path) as src:
            raster_bounds = src.bounds
            raster_box = box(*raster_bounds)
            
            for _, poly in self.polygons.iterrows():
                geom = poly.geometry
                # Skip invalid or empty geometries or those that do not intersect with the raster
                if geom is None or geom.is_empty or not geom.intersects(raster_box):
                    continue
                
                window = from_bounds(*geom.bounds, transform=src.transform)
                # Skip windows that are completely outside the raster bounds
                if window.width <= 0 or window.height <= 0:
                    continue
                
                data = src.read(window=window)
                
                # Skip if data is empty or all NaN
                if data.size == 0 or np.isnan(data).all():
                    continue
                
                # Collect necessary attributes from the polygon
                attributes = {
                    'GlobalID': str(poly['GlobalID']),  # Convert GlobalID to string to ensure consistency
                    # Add other attributes here as needed, e.g., 'species': poly['species']
                }
                
                # Attach attributes to the data chunk
                for i in range(data.shape[0]):  # Assuming data is [bands, rows, cols]
                    for row in range(data.shape[1]):
                        for col in range(data.shape[2]):
                            all_data.append({
                                'pixel_value_band_{}'.format(i+1): data[i, row, col],
                                'row': row + int(window.row_off),
                                'col': col + int(window.col_off),
                                **attributes
                            })
        
        return pd.DataFrame(all_data)



gpkg_path = 'Datasets/niwot_aop_polygons_2023_12_8_23_analysis_ready_half_diam.gpkg'
existing_raster_path = "NIWOT_calibration_flight_08_2020/NEON_D13_NIWO_DP1_20200801_161441_reflectance/NEON_D13_NIWO_DP1_20200801_161441_reflectanceNEON_D13_NIWO_DP1_20200801_161441_reflectance__envi"
processor = ENVIProcessor(existing_raster_path, gpkg_path)
df_polygons = processor.extract_data_by_polygons()
df_polygons

,pixel_value_band_1,row,col,GlobalID,pixel_value_band_2,pixel_value_band_3,pixel_value_band_4,pixel_value_band_5,pixel_value_band_6,pixel_value_band_7,...,pixel_value_band_417,pixel_value_band_418,pixel_value_band_419,pixel_value_band_420,pixel_value_band_421,pixel_value_band_422,pixel_value_band_423,pixel_value_band_424,pixel_value_band_425,pixel_value_band_426
0,739.0,1222,554,{E9346797-777A-4D43-BD01-02A511C57DAA},NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,300.0,1222,555,{E9346797-777A-4D43-BD01-02A511C57DAA},NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,26.0,1223,554,{E9346797-777A-4D43-BD01-02A511C57DAA},NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,429.0,1223,555,{E9346797-777A-4D43-BD01-02A511C57DAA},NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,383.0,1224,554,{E9346797-777A-4D43-BD01-02A511C57DAA},NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294361,NaN,792,551,{56551F70-2BB6-4C7A-80A4-99CF53CD2EC3},NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,815.0
294362,NaN,792,552,{56551F70-2BB6-4C7A-80A4-99CF53CD2EC3},NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.0
294363,NaN,793,550,{56551F70-2BB6-4C7A-80A4-99CF53CD2EC3},NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,461.0
294364,NaN,793,551,{56551F70-2BB6-4C7A-80A4-99CF53CD2EC3},NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,786.0


In [9]:
layer_columns = [f'pixel_value_band_{i}' for i in range(1, 426)]
df_polygons = df_polygons.replace(-9999.0, np.nan)
# Drop rows where all values in the specified columns are NaN
df_cleaned_polygons = df_polygons.dropna(subset=layer_columns, how='all')
df_cleaned_polygons

,pixel_value_band_1,row,col,GlobalID,pixel_value_band_2,pixel_value_band_3,pixel_value_band_4,pixel_value_band_5,pixel_value_band_6,pixel_value_band_7,...,pixel_value_band_417,pixel_value_band_418,pixel_value_band_419,pixel_value_band_420,pixel_value_band_421,pixel_value_band_422,pixel_value_band_423,pixel_value_band_424,pixel_value_band_425,pixel_value_band_426
0,739.0,1222,554,{E9346797-777A-4D43-BD01-02A511C57DAA},NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,300.0,1222,555,{E9346797-777A-4D43-BD01-02A511C57DAA},NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,26.0,1223,554,{E9346797-777A-4D43-BD01-02A511C57DAA},NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,429.0,1223,555,{E9346797-777A-4D43-BD01-02A511C57DAA},NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,383.0,1224,554,{E9346797-777A-4D43-BD01-02A511C57DAA},NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294355,NaN,792,551,{56551F70-2BB6-4C7A-80A4-99CF53CD2EC3},NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,823.0,NaN
294356,NaN,792,552,{56551F70-2BB6-4C7A-80A4-99CF53CD2EC3},NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,166.0,NaN
294357,NaN,793,550,{56551F70-2BB6-4C7A-80A4-99CF53CD2EC3},NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,339.0,NaN
294358,NaN,793,551,{56551F70-2BB6-4C7A-80A4-99CF53CD2EC3},NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,491.0,NaN
